In [203]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
import pandas as pd
import numpy as np
from operator import itemgetter, attrgetter

In [119]:
def load_data(data_path):
    """
    키워드 - word2vec 딕셔너리
    """
    zerovec = np.zeros((20,), dtype=np.float32)
    dic = {}
    with open(data_path, 'rt', encoding='utf8') as csvfile:
        reader = csv.reader(csvfile)
        w2c_dic = {}
        for row in reader:
            key=row[0]
            vec=np.array(list(map(lambda x:float(x), row[1:-1])), dtype=np.float32)
            if np.allclose(vec, zerovec) != True:
                dic[key]=vec
    return dic

In [120]:
#
dic_0940 = load_data(r'd:\WMIND\temp\data\result\model_1534375850586.csv\part-00000')
dic_0955 = load_data(r'd:\WMIND\temp\data\result\model_1534379450653.csv\part-00000')

In [116]:
dic_0955['여름']

array([-2.1750903, 11.656229 ,  0.6676253, 21.856644 , -1.9875165,
       13.427332 , 10.398073 ,  1.8186046,  5.6832595,  0.7159476,
       -3.6526413,  6.9150577, -3.230278 ,  5.8001475, -7.654966 ,
       10.17702  ,  3.525654 ,  5.1598554,  9.885317 ,  1.6343417],
      dtype=float32)

In [123]:
dic_0940['여름']

array([  7.8301716 ,  13.248194  , -14.633035  ,   0.37966922,
        -5.387335  ,   5.110036  ,   4.6691194 ,   6.516449  ,
        15.045565  ,  10.236552  ,   3.8216882 ,  -3.0212533 ,
        -1.6769195 ,  -0.35369164,  -8.497376  ,  10.399048  ,
       -13.144906  ,  -3.8625247 ,   2.724656  ,  -5.5872793 ],
      dtype=float32)

## 0816_09_40.org 로 부터 구성

In [47]:
def read_history(data_path):
    with open(data_path, 'rt', encoding='utf8') as fs:
        doAction=False
        history = []
        while True:
            line = fs.readline()
            if not line: break
            if doAction:
                line = line.rstrip()
                if len(line) == 0: break
                data = line.split('\t')
                if len(data) == 6:
                    did = int(data[0])
                    history.append( (did,list(map(lambda x: x.strip(), data[-1].strip()[1:-1].split(',')))) )
            if line.startswith('행동 최신'):
                doAction=True   
    return history

In [53]:
def read_wepick(data_path):
    with open(data_path, 'rt', encoding='utf8') as fs:
        doAction=False
        history = []
        while True:
            line = fs.readline()
            if not line: break
            if doAction:
                line = line.rstrip()
                if len(line) == 0: break
                data = line.split('\t')
                if len(data) == 5:
                    did = int(data[0])
                    history.append( (did,list(map(lambda x: x.strip(), data[-1].strip()[1:-1].split(',')))) )
            if line.startswith('결과 리스트'):
                doAction=True   
    return history

In [56]:
history_0940 = read_history(r'd:\WMIND\temp\data\0816_09_40.org')
wepick_0940 = read_wepick(r'd:\WMIND\temp\data\0816_09_40.org')
history_0955 = read_history(r'd:\WMIND\temp\data\0816_09_55.org')
wepick_0955 = read_wepick(r'd:\WMIND\temp\data\0816_09_55.org')

## 단비님이 만들어준 딜피쳐를 활용

In [161]:
def read_history_features(data_path):
    with open(data_path, 'rt', encoding='utf8') as fs:
        doAction=False
        history = []
        while True:
            line = fs.readline()
            if not line: break
            if doAction:
                line = line.rstrip()
                if len(line) == 0: break
                data = line.split('\t')
                if len(data) == 2:
                    did = int(data[0])
                    vec = list(map(lambda a: float(a), 
                              map(lambda x: x.strip(), 
                                  data[-1].strip()[1:-1].split(',')
                                 )
                             ) 
                        )
                    history.append( (did, np.array(vec, dtype=np.float32) ) )
            if line.startswith('사용자 행동 unique 딜'):
                doAction=True   
    return history

In [165]:
def read_wepick_features(data_path):
    with open(data_path, 'rt', encoding='utf8') as fs:
        doAction=False
        wepicks = []
        while True:
            line = fs.readline()
            if not line: break
            if doAction:
                line = line.rstrip()
                if len(line) == 0: break
                data = line.split('\t')
                if len(data) == 2:
                    did = int(data[0])
                    vec = list(map(lambda a: float(a), 
                              map(lambda x: x.strip(), 
                                  data[-1].strip()[1:-1].split(',')
                                 )
                             ) 
                        )
                    wepicks.append( (did, np.array(vec, dtype=np.float32) ) )
            if line.startswith('위픽 리스트 딜'):
                doAction=True   
    return wepicks

In [179]:
def calc_centroid(datas, strategy='average'):
    n = len(datas)
    centroid = np.zeros((20,))
    for did, vec in datas:
        centroid += vec
    return centroid / n

In [210]:
def calc_original_rank(cenroid, datas):
    """
    쿼리와, 각 data간 유사도를 재고, 정렬    
    """
    scores = []
    for did, vec in datas:
        scores.append( (did, np.dot(centroid, vec) / np.linalg.norm(centroid) / np.linalg.norm(vec) ) )
#         print(centroid, vec)
#         print(np.dot(centroid, vec) / np.linalg.norm(did) , np.linalg.norm(vec))
#         break
    return sorted(scores, key=itemgetter(1), reverse=True)

In [164]:
history_4000 = read_history_features(r'd:\WMIND\temp\data\20180816_4000_deal2vec.txt')
history_4003 = read_history_features(r'd:\WMIND\temp\data\20180816_4003_deal2vec.txt')

In [172]:
wepick_4000 = read_wepick_features(r'd:\WMIND\temp\data\20180816_4000_deal2vec.txt')
wepick_4003 = read_wepick_features(r'd:\WMIND\temp\data\20180816_4003_deal2vec.txt')

In [212]:
centroid_4000 = calc_centroid(history_4000)
centroid_4003 = calc_centroid(history_4003)

In [213]:
calc_original_rank(centroid_4000, wepick_4000)

[(3925584, 0.523868155977151),
 (3929546, 0.523868155977151),
 (3926878, 0.4995753045484816),
 (3917082, 0.4869143092858477),
 (3926552, 0.43565926189631576),
 (3925803, 0.4268817381130126),
 (3918668, 0.39444586426473877),
 (3925238, 0.36742748921653223),
 (3925745, 0.3558961915869726),
 (3929946, 0.3047584820338496),
 (1998145, 0.2661902885863478),
 (2205335, 0.23922762095106592),
 (3680030, 0.1987761649446484),
 (3921091, 0.1936548438496343),
 (3005018, 0.17130419289319326),
 (3927051, 0.1558177862795168),
 (1657902, 0.15117004782512689),
 (3126922, 0.14624592800606695),
 (2277736, 0.1460655803097252),
 (3928179, 0.13816804682940353),
 (3909664, 0.13816435439218874),
 (1355945, 0.12277758231345041),
 (3910546, 0.12085660655727112),
 (3921747, 0.11728783707996089),
 (3927178, 0.11453792038408157),
 (3922962, 0.11428991159797096),
 (3914261, 0.10766191274102457),
 (3925721, 0.10072634548269523),
 (1832247, 0.07981208068614554),
 (3919555, 0.06451065830765916),
 (3177907, 0.06331628290

In [214]:
calc_original_rank(centroid_4003, wepick_4003)

[(3921091, 0.3536476339119444),
 (3680030, 0.27973187397113264),
 (3921272, 0.26916813912999904),
 (2205335, 0.20937268303851703),
 (3926552, 0.20657748074452678),
 (1338912, 0.14794438409131652),
 (1832247, 0.12771385374281086),
 (1759162, 0.11971617159689793),
 (3177907, 0.11018993194233094),
 (1657902, 0.10323532162357604),
 (3921747, 0.10257546524145991),
 (3929946, 0.10212876749665736),
 (3005018, 0.0948630593709246),
 (3126922, 0.09357738857897391),
 (1923440, 0.09171755927396734),
 (3925745, 0.07231779512565775),
 (1355945, 0.05686314222445048),
 (2277736, 0.04770422945805973),
 (3927051, 0.043812711852870646),
 (1998145, 0.042399363285394474),
 (3272368, 0.0307290600550405),
 (3925721, 0.02327864801484646),
 (1358946, 0.014812443509797095),
 (3925803, 0.010281584739950765),
 (2853271, 0.0036193306754160257),
 (3037789, 0.0006235718832975058),
 (3386905, -0.003450697918112277),
 (1728389, -0.013596371178460796),
 (3914261, -0.026951705200763114),
 (3928179, -0.03150271146838257)

In [125]:
def get_w2vec_feature(keywords, dic, use_unit_length=False, do_final_normalize=True):
    avg_pool_vec = np.zeros((20,))
    n = 0
    for keyword in keywords:
        if keyword in dic:
            vec = dic[keyword]
#             print(keyword, vec)
            if use_unit_length:
                vec /= np.sqrt(np.sum(vec**2))
            avg_pool_vec += vec
            n += 1
    if do_final_normalize:
#         avg_pool_vec /= np.sqrt(np.sum(avg_pool_vec**2))
        avg_pool_vec /= n
    return avg_pool_vec

In [85]:
def gather_features(list_did_words, dic):
    result_dic = {}
    for did, words in list_did_words:
        result_dic[did] = get_w2vec_feature(words, dic)
    return result_dic

In [88]:
def calc_centroid(dic):
    n = len(dic)
    vec = np.zeros((20,))
    for val in dic.values():
        vec += val
    return vec / n

In [91]:
def make_unitlength(vec):
    return vec / np.sqrt(np.sum(vec**2))

In [108]:
get_w2vec_feature(history_0940[0][1], dic_0940)

array([-0.1377285 , -0.26697709, -0.16585805,  0.14383845, -0.22717078,
       -0.40002458, -0.2382803 ,  0.01891664, -0.06828908, -0.4241687 ,
       -0.0666227 , -0.01874852, -0.2362409 ,  0.1191254 , -0.02492022,
       -0.1193964 ,  0.07401314,  0.21216158,  0.05964968,  0.17014683])

In [126]:
get_w2vec_feature(wepick_0940[0][1], dic_0940)

array([  7.96881423, -15.69272117,   0.47264806,   5.34064629,
        -6.51507795,   4.87499367, -19.03162728,   7.55737817,
         9.68323777,   2.75139078,   2.86543947,  -2.78664333,
        -8.65409099,   1.81316907, -13.01808817,  20.26527736,
       -16.41763422,  19.58289178,  -2.33877682,  -9.35977163])

In [103]:
x = calc_centroid(gather_features( history_0940, dic_0940))

In [104]:
make_unitlength( calc_centroid(gather_features( history_0940, dic_0940)) )

array([-0.19968645, -0.19608969, -0.09171642,  0.09476951, -0.23625785,
       -0.48686596, -0.37358235,  0.06122241, -0.06941806, -0.36771063,
        0.17648013, -0.04432764, -0.33517561,  0.07835231, -0.24672328,
        0.07962579, -0.09784611,  0.31471156,  0.00939191,  0.02393308])

In [134]:
my = gather_features(history_0955, dic_0955)

In [135]:
my

{1318568: array([  8.3881347 ,  -6.58152156,   0.56447959, -17.02123139,
        -13.39107309,   5.16036158,  -4.27678948,   8.45737702,
         -3.47249573,   6.14851319,  14.38284023,   7.14663591,
         -6.6709235 ,  -5.21388376,   6.4832806 ,   1.64906898,
         12.07407006, -14.65509638,  -2.17780701,  -1.07846131]),
 2577177: array([  4.48177732, -22.64745792,  15.62512451,  -4.6136963 ,
         -6.99450439,   0.545422  , -22.4140969 ,  24.75354513,
         -4.54675408, -10.4760142 ,  18.93518353,   0.95213539,
        -11.22067432,  40.75714567,  32.01246496,  -3.97705163,
         19.37084097,  22.23507481,   5.34839206,  25.46153029]),
 2742984: array([ -2.39299648, -38.28628874,   8.79081784, -13.84436182,
          0.10570662,   7.09678754,   3.25021691,  -8.89260777,
          3.7160199 ,   8.47871016,  19.88630676,   6.24242075,
        -13.01542041, -14.86757346,  13.78883441,   4.70188895,
         17.93412092, -33.13398708,  -3.28943749,  -5.64461662]),
 340342